In [43]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
# Import the train_test_learn module
from sklearn.model_selection import train_test_split
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [44]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
data = Path('Resources/lending_data.csv')
lending_df = pd.read_csv(data)

# Review the DataFrame
display(lending_df.head())
display(lending_df.tail())

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
77531,19100.0,11.261,86600,0.653580,12,2,56600,1
77532,17700.0,10.662,80900,0.629172,11,2,50900,1
77533,17600.0,10.595,80300,0.626401,11,2,50300,1
77534,16300.0,10.068,75300,0.601594,10,2,45300,1
77535,15600.0,9.742,72300,0.585062,9,2,42300,1


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [45]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_df['loan_status']

# Separate the X variable, the features
X = lending_df.drop(columns=['loan_status'])

In [46]:
# Review the y variable Series
y.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [47]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [48]:
# Check the balance of our target values
y.value_counts()

loan_status
0    75036
1     2500
Name: count, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [49]:
# Import the train_test_learn module(already imported)
# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    stratify=y,
                                                    random_state=1, 
                                                    )

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [50]:
# Import the LogisticRegression module from SKLearn(already imported)

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs',
                                max_iter = 200,
                                random_state=1)

# Fit the model using training data
lr_model = classifier.fit(X_train, y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [51]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
36831,0,0
75818,0,1
36563,0,0
13237,0,0
43292,0,0
...,...,...
38069,0,0
36892,0,0
5035,0,0
40821,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [52]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, predictions)

0.9442676901753825

In [53]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,18679,80
Actual 1,67,558


In [54]:
# Print the classification report for the model
print("Classification Report")
target_names = ["healthy loan","high-risk loan"]
print(classification_report(y_test, predictions, target_names = target_names))

Classification Report
                precision    recall  f1-score   support

  healthy loan       1.00      1.00      1.00     18759
high-risk loan       0.87      0.89      0.88       625

      accuracy                           0.99     19384
     macro avg       0.94      0.94      0.94     19384
  weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** In summary, the logistic regression model performs exceptionally well for "healthy loans" with perfect precision, recall, and F1-score. It also performs quite well for "high-risk loans" with high precision, recall, and F1-score. Overall, it's a strong model for this classification task, but there is a slightly better performance for "healthy loans."

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [55]:
# Import the RandomOverSampler module form imbalanced-learn (already imported)

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
model_ros = RandomOverSampler(random_state=1)
# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = model_ros.fit_resample(X_train, y_train)

In [56]:
# Count the distinct values of the resampled labels data
from collections import Counter
print(Counter(X_ros_model))
print(Counter(y_ros_model))
print(f"The y_ros_model resampled data is equivalently split")

Counter({'loan_size': 1, 'interest_rate': 1, 'borrower_income': 1, 'debt_to_income': 1, 'num_of_accounts': 1, 'derogatory_marks': 1, 'total_debt': 1})
Counter({0: 75036, 1: 75036})
The y_ros_model resampled data is equivalently split


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [57]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier_ros = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
classifier_ros.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
predictions_ros = classifier_ros.predict(X_test)
pd.DataFrame({"Prediction": predictions_ros, "Actual": y_test})

,Prediction,Actual
36831,0,0
75818,1,1
36563,0,0
13237,0,0
43292,0,0
...,...,...
38069,0,0
36892,0,0
5035,0,0
40821,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [58]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, predictions_ros)

0.9959744975744975

In [59]:
# Generate a confusion matrix for the model
matrix = confusion_matrix(y_test, predictions_ros)
matrix_df = pd.DataFrame(
   matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
display(matrix_df) 

,Predicted 0,Predicted 1
Actual 0,18668,91
Actual 1,2,623


In [60]:
# Print the classification report for the model
print("ROS Classification Report")
target_names = ["healthy loan","high-risk loan"]
print(classification_report(y_test, predictions_ros, target_names = target_names))

ROS Classification Report
                precision    recall  f1-score   support

  healthy loan       1.00      1.00      1.00     18759
high-risk loan       0.87      1.00      0.93       625

      accuracy                           1.00     19384
     macro avg       0.94      1.00      0.96     19384
  weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** How accurate the model is when it predicts a particular type of loan. For "healthy loans," the model is 100% accurate in its predictions, which means it only says a loan is healthy when it's really healthy. For "high-risk loans," it's 87% accurate.

Recall: How good the model is at finding all the loans of a certain type. In this case, it's really good at finding both healthy and high-risk loans, with a score of 100% for both.

F1-score: Balance between precision and recall. For "healthy loans," it's perfect at 1.00, but for "high-risk loans," it's a bit lower at 0.93, indicating a bit of a trade-off between accuracy and completeness.

Support: Tells us how many of each type of loan there are in the dataset. There are 18,759 "healthy loans" and 625 "high-risk loans."

Accuracy: The model gets 100% of the predictions correct for the entire dataset. It's really good at classifying loans.

Macro Average: This is an average of the precision, recall, and F1-score for both types of loans. It's 94% for precision, 100% for recall, and 96% for F1-score.

Weighted Average: Similar to macro average, but it considers the number of loans in each category. Since there are more "healthy loans," it's weighted more towards that. So, it's 100% for precision, 100% for recall, and 100% for F1-score.